# Scraping from BBC Urdu

## Importing the libraries

In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np
import pickle

In [2]:
url='https://www.bbc.com/urdu'
r=requests.get(url)
htmlc=r.content

In [3]:
soup=BeautifulSoup(htmlc,'html.parser' )

nav_bars=soup.find_all('li', class_='bbc-1v9wiw8 e1lim4kn2')

links_list=['a','a','a','a','a','a','a','a']

In [4]:
index=0
for i in nav_bars:
    t=i.find('a')
    l='https://www.bbc.com' + t.get('href')
    links_list[index]=l
    index+=1

In [5]:
links_list.remove("https://www.bbc.com/urdu")
links_list.remove("https://www.bbc.com/urdu/media/video")

newlinks=[] #declare list to store the links of all (600) pages 

## Generating the links

In [6]:
#populate new list with links 
for i in links_list:
    newlinks.append(i)
    for l in range(99):
        ab=i + '/page/' + str(l+2)
        newlinks.append(ab)

## Scraping data from each link

In [8]:
bbc={}
indexe=0
for link in newlinks:
    indexe+=1
    print(indexe)
    source = requests.get(link).text
    soup = BeautifulSoup(source, 'lxml')
    articles = soup.find_all('li', class_='lx-stream__post-container')

    for article_ in articles:
       
        article_name=article_.article.header.div.h3.a.span.text
        
        try:
            more_info = article_.find('span', class_='lx-stream-related-story--cta-text').text
            more_info_link = article_.find('a', class_='qa-story-cta-link')['href']
            more_info_link='https://www.bbc.com/'+more_info_link
            source1 = requests.get(more_info_link).text
            soup1=BeautifulSoup(source1, 'lxml')
            inner_data = soup1.find_all('div', class_='bbc-4wucq3 e57qer20')
            p=''
            for data_ in inner_data:
                try:
                    p += data_.find('p').text
                    
                except Exception as e:
                    ay=2
        except Exception as e:
            ay=2
        bbc.update({article_name:p})
        f = open('bbc_urdu.pkl', 'wb')   # 'wb' instead 'w' for binary file
        pickle.dump(bbc, f, -1)       # -1 specifies highest binary protocol
        f.close()

1
2
3
4
5
